In [25]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
# from folium.plugins import FloatImage
import numpy as np
import json
from folium import FeatureGroup, LayerControl

In [26]:
st_abbrev = json.load(open("st_abbrev.json",'r'))

In [27]:
curl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv" 
rurl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"
durl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
df = pd.read_csv(curl)
df = df.drop(columns = list(df.columns)[4:-1])
df = df[df["Country/Region"]=="US"]
df = df.rename(columns={list(df.columns)[-1]:"cases"})
df = df.rename(columns={"Province/State":"states"})
df = df[np.array([len(word.split(',')) for word in df['states'].tolist()])==1]
df = df[[x in st_abbrev.keys() for x in df.states]]
df["states"].replace(st_abbrev,inplace=True)

In [28]:
def process_df(url,st_abbrev):
    df = pd.read_csv(url)
    df = df.drop(columns = list(df.columns)[4:-1])
    df = df[df["Country/Region"]=="US"]
    df = df.rename(columns={list(df.columns)[-1]:"cases"})
    df = df.rename(columns={"Province/State":"states"})
    df = df[np.array([len(word.split(',')) for word in df['states'].tolist()])==1]
    df = df[[x in st_abbrev.keys() for x in df.states]]
    df["states"].replace(st_abbrev,inplace=True)
    return df

curr_df = process_df(curl,st_abbrev)
recovery_df = process_df(rurl,st_abbrev)
death_df = process_df(durl,st_abbrev)
curr_df.head()

,states,Country/Region,Lat,Long,cases
98,WA,US,47.4009,-121.4905,904
99,NY,US,42.1657,-74.9481,967
100,CA,US,36.1162,-119.6816,557
101,MA,US,42.2302,-71.5301,197
104,GA,US,33.0406,-83.6431,121


In [29]:
import requests
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
us_states = f'{url}/us-states.json'
US_Unemployment_Oct2012 = f'{url}/US_Unemployment_Oct2012.csv'

geo_json_data = json.loads(requests.get(us_states).text)

In [30]:
import branca.colormap as cm
linear = cm.LinearColormap(
    ['orange', 'red'],
    vmin=0, vmax=max(curr_df.cases)
)
# linear
# curr_df.cases
def my_color_function(feature):
    return linear(list(curr_df[curr_df["states"]==feature['id']].cases)[0])

In [31]:
covid_map = folium.Map(location=[curr_df['Lat'].mean(),curr_df['Long'].mean()],zoom_start=4,zoom_control=False)
states_feature = FeatureGroup(name="States Heatmap")
folium.GeoJson(geo_json_data,
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }
).add_to(states_feature)
states_feature.add_to(covid_map)
# cdf = curr_df
# for state in cdf['states'].unique():
#   if(cdf[cdf['states']==state].iloc[-1][-1]!=0):
#     folium.Marker([cdf[cdf['states']==state]['Lat'].values[0],cdf[cdf['states']==state]['Long'].values[0]],
#                 popup='<i>No. of confirmed cases: %s</i>'%(cdf[cdf['states']==state].iloc[-1][-1]) , \
#                   tooltip=state,icon=folium.Icon(color='red',icon_size=(30, 30))).add_to(covid_map)    
icon_create_function = '''
 function (cluster) {
 var childCount = cluster.getChildCount();
 var c = ' marker-cluster-';
 if (childCount < 0) {
   c += 'small';
 } 
 else if (childCount < 0) {
   c += 'medium';
 } 
 else {
   c += 'large';
 }

 return new L.DivIcon({ html: '<div><span>' + childCount + '</span></div>', 
  className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) });
 }
'''

feature_group = FeatureGroup(name='Confirmed Cases')
mc = MarkerCluster(icon_create_function=icon_create_function)

for row in df.itertuples():
    for _ in range(row.cases):
        mc.add_child(folium.Circle(location=[row.Lat, row.Long],radius=0,color='red'))


mc.add_to(feature_group)
feature_group.add_to(covid_map)
# url = 'https://raw.githubusercontent.com/nthakor/imghost/master/blueicon.png'
# FloatImage(url, bottom = 82, left=5).add_to(covid_map)

In [32]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SAMPLE_SPREADSHEET_ID = '1QmwkYAsWf6_4LP4-vTADqn55DePslfjNSgtJ770v0vk'
SAMPLE_RANGE_NAME = 'Centers'

def get_data_from_sheets():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])
    return pd.DataFrame.from_records(values,columns=values[0])
tloc = get_data_from_sheets()
tloc = tloc.drop(tloc.index[0])
tloc[["Latitute", "Longitude"]] = tloc[["Latitute", "Longitude"]].apply(pd.to_numeric)
tloc = tloc[tloc['Latitute'].notnull()]

In [33]:
from folium.plugins import BeautifyIcon
testing_feature = FeatureGroup(name='Testing Centers')
for row in tloc.itertuples():
    folium.Marker((row.Latitute,row.Longitude),popup = folium.Popup('<a href="%s" target="_blank"> %s </a>'%(row.URL,row._2)),icon=folium.Icon(color='blue')).add_to(testing_feature)
testing_feature.add_to(covid_map)
# LayerControl().add_to(covid_map)


In [12]:
# covid_map.save("covid_19_us.html")

In [36]:
import bs4
# with open("covid_19_us.html") as f:
#     txt = f.read()
soup = bs4.BeautifulSoup(covid_map.get_root().render(),'html.parser')
with open('map_title.html') as f:
    txt = f.read()
    title = bs4.BeautifulSoup(txt,'html.parser')
with open('map_bottom.html') as f:
    txt = f.read()
    bottom = bs4.BeautifulSoup(txt,'html.parser')
title_command = title.div
bottom_command = bottom.div
soup.body.div.insert_after(title_command)
soup.body.div.insert_after(bottom_command)
soup.body

<body>
<div class="folium-map" id="map_fb2896052fc24e18aeff1d3b18cda8c6"></div><div class="grid-container-bottom" id="bottom_div">
<div>
<p>Confirmed cases data obtained from <a href="https://coronavirus.jhu.edu/" target="_black">coronavirus.jhu.edu
        </a></p>
</div>
<div></div>
</div><div class="grid-container-top" id="top_div">
<div>
<a href="https://www.sokat.co/" target="_blank"><img alt="float_image" class="responsive" id="float_image_e695752ea9b14073a71fa159ee72b319" src="https://raw.githubusercontent.com/nthakor/imghost/master/blueicon.png" style="padding-left:0%"/>
</a>
<p style="word-break: break-all font-size:0.2vw">Please email - <a href="mailto:testing@sokat.co">testing@sokat.co</a> new locations</p>
</div>
<div>
<h2>Location of Drive-Thru COVID19 Testing Sites across America</h2>
<h4>Blue Flags mark drive-thru testing sites and Orange Circles display the number of confirmed cases</h4>
</div>
</div>
</body>

In [37]:
map_style = bs4.BeautifulSoup('<link rel="stylesheet" href="mapStyle.css"/>','html.parser')
soup.head.style.insert_after(map_style)
soup.head

<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<script>
            L_NO_TOUCH = false;
            L_DISABLE_3D = false;
        </script>
<script src="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.js"></script>
<script src="https://code.jquery.com/jquery-1.12.4.min.js"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>
<link href="https://cdn.jsdelivr.net/npm/leaflet@1.5.1/dist/leaflet.css" rel="stylesheet"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css" rel="stylesheet"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css" rel="stylesheet"/>
<link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css" rel="stylesheet"/>
<link href="https://cdnjs.cloudflare.com/ajax/libs/Leafle

In [ ]:
# # nt = soup.new_tag('p',id="float_text")
# # nt.string = "Please Email new locations"
# # nt.append(soup.new_tag('a',href="anc"))
# # nt
# nt = bs4.BeautifulSoup('<p id="float_text">Please email - <a href="mailto:tests@sokat.co">test@sokat.co</a> new locations</p>')
# st = soup.new_tag('style', type='text/css')
# st.append('#float_text {position: absolute;bottom: 78%;left: 5%;z-index: 999999;}')
# soup.head.append(st)
# soup.a.insert_after(nt.p)
# title = bs4.BeautifulSoup()


In [ ]:
type(soup.find_all('script')[-1])

In [38]:
with open("covid_19_us.html", "w") as file:
    file.write(str(soup))

In [ ]:
print(soup.body.prettify())